In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('../../data/dataset_from_json.csv')
df.head()

,words,sentence #,tag
0,CPT,0,Incoterms
1,LAHORE,0,Incoterms
2,AIRPORT,0,Incoterms
3,PAKISTAN,0,Incoterms
4,QTY,0,O


In [29]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 0
        self.data = df
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [30]:
getter = SentenceGetter(df)
getter.get_next()
print(getter.get_next())
sentences = getter.sentences

[('SERVICES', 'GoodsDescription'), ('AND', 'GoodsDescription'), ('MAINTENANCE', 'GoodsDescription'), ('AS', 'O'), ('PER', 'O'), ("BENEFICIARY'S", 'O'), ('PROFORMA', 'O'), ('INVOICE', 'O'), ('NO.', 'O'), ('201912Q016H', 'O'), ('DATED', 'O'), ('05.12.2019', 'O')]


In [31]:
words = list(set(df["words"].values))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(df['tag'].values))
n_tags = len(tags)

max_len = 300
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [32]:
from keras_preprocessing.sequence import pad_sequences

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
from turtle import shape
from tensorflow.keras.layers import Input
from keras.models import Model
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, TimeDistributed, LSTM, Bidirectional

input = Input(shape=(max_len,))
x = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)  # 50-dim embedding

x = Conv1D(32, 3, activation='relu', padding='same')(x)
# # x = MaxPooling1D(3)(x)

x = Conv1D(64, 3, activation='relu', padding='same')(x)
# # x = MaxPooling1D(3)(x)

x = Conv1D(128, 3, activation='relu', padding='same')(x)
# x = MaxPooling1D(3)(x)

x = Conv1D(300, 3, activation='relu', padding='same')(x)
# x = LSTM(units=200, return_sequences=True)(x) 
# x = Dense(300, activation="relu")(x)
x = Dense(400, activation="relu")(x)
# x = LSTM(units=200, return_sequences=True)(x) 


out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
# x = Dense(n_tags, activation='relu')(x)
# out = Dense(n_tags, activation="softmax")(x)   # for k classes

model = Model(input, out)

In [36]:
from tensorflow.keras.metrics import AUC, Precision, Recall

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy", AUC(), Precision(), Recall()])
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 300)]             0         
                                                                 
 embedding_7 (Embedding)     (None, 300, 50)           468000    
                                                                 
 conv1d (Conv1D)             (None, 300, 32)           4832      
                                                                 
 conv1d_1 (Conv1D)           (None, 300, 64)           6208      
                                                                 
 conv1d_2 (Conv1D)           (None, 300, 128)          24704     
                                                                 
 conv1d_3 (Conv1D)           (None, 300, 300)          115500    
                                                                 
 dense_11 (Dense)            (None, 300, 400)          1204

In [37]:
history = model.fit(X_train, np.array(y_train), batch_size=64, epochs=10, validation_split=0.2, verbose=1)

Epoch 1/10


2022-10-03 10:16:02.823377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13XFamilyCommandBuffer: 0x2e7225740>
    label = <none> 
    device = <AGXG13XDevice: 0x14fe01a00>
        name = Apple M1 Pro 
    commandQueue = <AGXG13XFamilyCommandQueue: 0x2925ca200>
        label = <none> 
        device = <AGXG13XDevice: 0x14fe01a00>
            name = Apple M1 Pro 
    retainedReferences = 1


17/17 [==============================] - ETA: 0s - loss: 0.5462 - accuracy: 1.1702 - auc_5: 0.9733 - precision_5: 0.9641 - recall_5: 0.8090

2022-10-03 10:16:15.307560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 16s 832ms/step - loss: 0.5462 - accuracy: 1.1702 - auc_5: 0.9733 - precision_5: 0.9641 - recall_5: 0.8090 - val_loss: 0.1963 - val_accuracy: 1.2631 - val_auc_5: 0.9986 - val_precision_5: 0.9970 - val_recall_5: 0.9162
Epoch 2/10
17/17 [==============================] - 15s 885ms/step - loss: 0.1796 - accuracy: 1.2592 - auc_5: 0.9986 - precision_5: 0.9875 - recall_5: 0.9264 - val_loss: 0.1556 - val_accuracy: 1.2723 - val_auc_5: 0.9986 - val_precision_5: 0.9832 - val_recall_5: 0.9346
Epoch 3/10
17/17 [==============================] - 13s 774ms/step - loss: 0.1376 - accuracy: 1.2744 - auc_5: 0.9991 - precision_5: 0.9897 - recall_5: 0.9417 - val_loss: 0.1394 - val_accuracy: 1.2769 - val_auc_5: 0.9983 - val_precision_5: 0.9761 - val_recall_5: 0.9520
Epoch 4/10
17/17 [==============================] - 12s 701ms/step - loss: 0.0986 - accuracy: 1.2914 - auc_5: 0.9993 - precision_5: 0.9908 - recall_5: 0.9577 - val_loss: 0.0985 - val_accuracy: 1.2951 - va

KeyboardInterrupt: 